In [ ]:
import tensorflow as tf
import mitdeeplearning as mdl
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sport = tf.constant(5, tf.int16)
tf.rank(sport)

In [ ]:
sports  = tf.constant([1,2],tf.int16)
tf.rank(sports)

In [ ]:
numbers = tf.constant([3.141592, 1.414213, 2.71821], tf.float64)

In [ ]:
tf.rank(numbers)

In [ ]:
matrix = tf.constant([[1,2],[3,4]],tf.int16)

In [ ]:
tf.rank(matrix)


In [ ]:
print(matrix.shape)

In [ ]:
assert tf.rank(matrix).numpy() == 2

In [ ]:
images = tf.zeros([10,256,256,3],tf.int16)

In [ ]:
assert isinstance(images, tf.Tensor), "matrix must be a tf Tensor object"
assert tf.rank(images).numpy() == 4, "matrix must be of rank 4"

In [ ]:
assert tf.shape(images).numpy().tolist() == [10, 256, 256, 3], "matrix is incorrect shape"

In [ ]:
tf.rank(images)

In [ ]:
row_vector = matrix[1]

In [ ]:
print(row_vector)

In [ ]:
column_vector = matrix[:,1]


In [ ]:
print("`column_vector`: {}".format(column_vector.numpy()))


In [ ]:
#Computation on Tensors

In [ ]:
a = tf.constant(15)
b = tf.constant(61)
c1 = tf.add(a,b)
c2  = a  + b
print(c1)
print(c2)

In [ ]:
def func(a,b):
    c = tf.add(a,b)
    d = tf.subtract(b , 1)
    e = tf.multiply(c,d)
    return e

In [ ]:
a,b = 1.5,2.5
e_out = func(a,b)
print(e_out)

In [ ]:
sample = matrix -1 

In [ ]:
sample

In [ ]:
## defining a network Layer

In [ ]:
class OurDenseLayer(tf.keras.layers.Layer):
    def __init__(self, n_output_nodes):
        super(OurDenseLayer,self).__init__()
        self.n_output_nodes = n_output_nodes
    
    def build(self,input_shape):
        d  =  int(input_shape[-1])
        self.W = self.add_weight("Weight", shape = [d, self.n_output_nodes])
        self.b = self.add_weight("Bias", shape=[1,self.n_output_nodes])
    
    
    def call(self,x):
        z = tf.matmul(x,self.W) + self.b   
        y = tf.sigmoid(z)
        return y
    

In [ ]:
tf.random.set_seed(1)
layer = OurDenseLayer(3)
layer.build((1,2))
x_input = tf.constant([[1,2.]], shape=(1,2))
y = layer.call(x_input)

# test the output!
print(y.numpy())

In [ ]:
print(layer.W)

In [ ]:
mdl.lab1.test_custom_dense_layer_output(y)


In [ ]:
# Sequence of layers

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
n_output_nodes = 3
model = Sequential()
#Creating Dense Layer
dense_layer = Dense(n_output_nodes)
model.add(dense_layer)













In [ ]:
x_input = tf.constant([[1,2.]], shape=(1,2))





'''TODO: feed input into the model and predict the output!'''
model_output = model(x_input).numpy()
print("Number of weights after calling the model:", len(model.weights))  # 6




# model_output = # TODO
print(model_output)

In [ ]:
biases = model.layers[0].get_weights()[1]
weights = model.layers[0].get_weights()[0]



In [ ]:
biases

In [ ]:
weights

In [ ]:
#using Model as base class 


In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense


In [ ]:
class SubclassModel(tf.keras.Model):

  # In __init__, we define the Model's layers
  def __init__(self, n_output_nodes):
    super(SubclassModel, self).__init__()
    '''TODO: Our model consists of a single Dense layer. Define this layer.''' 
    self.dense_layer = Dense(n_output_nodes)
    
    
    
    
  def call(self, inputs,isidentity =False):
    
    return self.dense_layer(inputs)

In [ ]:
layer = SubclassModel(3)

In [ ]:
x_input = tf.constant([[1,2.]], shape=(1,2))

In [ ]:
y = layer(x_input)

In [ ]:
print(y)

In [ ]:
layer.get_weights()

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense

class IdentityModel(tf.keras.Model):

  # As before, in __init__ we define the Model's layers
  # Since our desired behavior involves the forward pass, this part is unchanged
  def __init__(self, n_output_nodes):
    super(IdentityModel, self).__init__()
    self.dense_layer = tf.keras.layers.Dense(n_output_nodes, activation='sigmoid')

  '''TODO: Implement the behavior where the network outputs the input, unchanged, under control of the isidentity argument.'''
  def call(self, inputs, isidentity=False):
    x = self.dense_layer(inputs)
    if isidentity: # TODO
      return inputs # TODO
    return x
  
  # def call(self, inputs, isidentity=False):
    # TODO

In [ ]:
_output_nodes = 3
model = IdentityModel(n_output_nodes)

x_input = tf.constant([[1,2.]], shape=(1,2))
'''TODO: pass the input into the model and call with and without the input identity option.'''
out_activate = model.call(x_input) # TODO
# out_activate = # TODO
out_identity = model.call(x_input, isidentity=True) # TODO
# out_identity = # TODO

print("Network output with activation: {}; network identity output: {}".format(out_activate.numpy(), out_identity.numpy()))

In [ ]:
#Differnatiation


In [ ]:
x = tf.Variable(3.0)
# Initiate the gradient tape
with tf.GradientTape() as tape:
  # Define the function
  y = x * x
# Access the gradient -- derivative of y with respect to x
dy_dx = tape.gradient(y, x)


In [ ]:
### Function minimization with automatic differentiation and SGD ###

# Initialize a random value for our initial x
x = tf.Variable([tf.random.normal([1])])
print("Initializing x={}".format(x.numpy()))

learning_rate = 1e-2 # learning rate for SGD
history = []
# Define the target value
x_f = 4

# We will run SGD for a number of iterations. At each iteration, we compute the loss, 
#   compute the derivative of the loss with respect to x, and perform the SGD update.
for i in range(1000):
  with tf.GradientTape() as tape:
    '''TODO: define the loss as described above'''
    loss = (x - x_f)**2 # "forward pass": record the current loss on the tape
    # loss = # TODO

  # loss minimization using gradient tape
  grad = tape.gradient(loss, x) # compute the derivative of the loss with respect to x
  new_x = x - learning_rate*grad # sgd update
  x.assign(new_x) # update the value of x
  history.append(x.numpy()[0])

# Plot the evolution of x as we optimize towards x_f!
plt.plot(history)
plt.plot([0, 500],[x_f,x_f])
plt.legend(('Predicted', 'True'))
plt.xlabel('Iteration')
plt.ylabel('x value')

In [ ]:
x